In [5]:
import pandas as pd
import numpy as np

data_transactions = {
    'transaction_id': [1, 2, 3, 4, 5, 6, 7, 8],
    'client_id': [101, 102, 101, 103, 102, 101, 103, 102],
    'transaction_date': ['2024-01-15', '2024-02-20', '2024-03-10', '2024-04-05', '2024-05-12', '2023-12-25', '2024-06-01', '2024-07-18'],
    'amount': ['1000.50', '200.75', '150.00', '500.25', '300.50', '1200.00', '700.00', '400.00'],
    'currency': ['USD', 'EUR', 'USD', 'USD', 'EUR', 'USD', 'USD', 'EUR'],
    'transaction_type': ['deposit', 'withdrawal', 'deposit', 'withdrawal', 'deposit', 'deposit', 'withdrawal', 'withdrawal']
}

data_clients = {
    'client_id': [101, 102, 103],
    'client_name': ['Иван Иванов', 'Петр Петров', 'Сидор Сидоров'],
    'registration_date': ['2023-12-01', '2024-01-10', '2024-02-15'],
    'client_status': ['active', 'active', 'inactive']
}

transactions = pd.DataFrame(data_transactions)
clients = pd.DataFrame(data_clients)

transactions['amount'] = pd.to_numeric(transactions['amount'])

transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'])
clients['registration_date'] = pd.to_datetime(clients['registration_date'])

print("\nПосле преобразования типов:")
print(transactions.dtypes)
print(transactions)


После преобразования типов:
transaction_id               int64
client_id                    int64
transaction_date    datetime64[ns]
amount                     float64
currency                    object
transaction_type            object
dtype: object
   transaction_id  client_id transaction_date   amount currency  \
0               1        101       2024-01-15  1000.50      USD   
1               2        102       2024-02-20   200.75      EUR   
2               3        101       2024-03-10   150.00      USD   
3               4        103       2024-04-05   500.25      USD   
4               5        102       2024-05-12   300.50      EUR   
5               6        101       2023-12-25  1200.00      USD   
6               7        103       2024-06-01   700.00      USD   
7               8        102       2024-07-18   400.00      EUR   

  transaction_type  
0          deposit  
1       withdrawal  
2          deposit  
3       withdrawal  
4          deposit  
5          deposi

In [7]:
active_clients = clients[clients['client_status'] == 'active']['client_id']

transactions_2024 = transactions[
    (transactions['transaction_date'].dt.year == 2024) &
    (transactions['client_id'].isin(active_clients))
]
print("\nТранзакции активных клиентов за 2024 год:")
print(transactions_2024)

result = transactions_2024.groupby(['client_id', 'transaction_type']).agg(
    total_amount=('amount', 'sum'),
    average_amount=('amount', 'mean'),
    transaction_count=('transaction_id', 'count'),
    last_transaction_date=('transaction_date', 'max')
).reset_index()

print("\nПосле группировки:")
print(result)


Транзакции активных клиентов за 2024 год:
   transaction_id  client_id transaction_date   amount currency  \
0               1        101       2024-01-15  1000.50      USD   
1               2        102       2024-02-20   200.75      EUR   
2               3        101       2024-03-10   150.00      USD   
4               5        102       2024-05-12   300.50      EUR   
7               8        102       2024-07-18   400.00      EUR   

  transaction_type  
0          deposit  
1       withdrawal  
2          deposit  
4          deposit  
7       withdrawal  

После группировки:
   client_id transaction_type  total_amount  average_amount  \
0        101          deposit       1150.50         575.250   
1        102          deposit        300.50         300.500   
2        102       withdrawal        600.75         300.375   

   transaction_count last_transaction_date  
0                  2            2024-03-10  
1                  1            2024-05-12  
2                  2

In [8]:
result = result.merge(clients[['client_id', 'client_name']], on='client_id', how='left')

result = result[['client_id', 'client_name', 'transaction_type',
                 'total_amount', 'average_amount',
                 'transaction_count', 'last_transaction_date']]

result = result.sort_values('total_amount', ascending=False).reset_index(drop=True)

result['total_amount'] = result['total_amount'].round(2)
result['average_amount'] = result['average_amount'].round(2)
result['last_transaction_date'] = result['last_transaction_date'].dt.strftime('%Y-%m-%d')

print("ИТОГОВЫЙ ОТЧЕТ")
print(result)

ИТОГОВЫЙ ОТЧЕТ
   client_id  client_name transaction_type  total_amount  average_amount  \
0        101  Иван Иванов          deposit       1150.50          575.25   
1        102  Петр Петров       withdrawal        600.75          300.38   
2        102  Петр Петров          deposit        300.50          300.50   

   transaction_count last_transaction_date  
0                  2            2024-03-10  
1                  2            2024-07-18  
2                  1            2024-05-12  
